# Trabalho Final

**Aluno**: Carlos Alberto Rocha Cardoso

**Matrícula**: 96983



## 05 - Tratamento da base *experiencia_trabalho.csv*

Essa base descreve a experiência profissional dos funcionários, representados pela coluna chave **perfil_id**, apresentando a companhia (coluna **company**), localização (coluna **location**), o cargo (coluna **position**) e o período de início e término da experiência profissional (coluna **date**). Podem existir mais de um registro por **perfil_id**, no caso do funcionário possuir experiências profissionais diversas.

Todos os 111 funcionários constam nessa base, porém o funcionário perfil_id = 108 não constará na base resultante do processo de tratamento e consolidação, uma vez que seu único registro de experiência não possui o período de início e término. Dessa forma 110 funcionários terão suas experiências profissionais consolidadas nas bases processadas. 

In [2]:
import numpy as np
import pandas as pd
from unicodedata import normalize

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

experiencia = pd.read_csv("bases/experiencia_trabalho.csv")

# Análise inicial
print("Nº de registros:", len(experiencia))
print("Nº de valores únicos de perfil_id:", experiencia.perfil_id.nunique())
print("Nº de valores únicos de company:", experiencia.company.nunique())
print("Nº de valores únicos de location:", experiencia.location.nunique())
print("Nº de valores únicos de position:", experiencia.position.nunique())
print("Nº de valores vazios de company:", experiencia.company.isnull().sum())
print("Nº de valores vazios de date:", experiencia.date.isnull().sum())
print("Nº de valores vazios de location:", experiencia.location.isnull().sum())
print("Nº de valores vazios de position:", experiencia.position.isnull().sum())

experiencia.describe(include="all")

idx_perfis = experiencia.perfil_id

Nº de registros: 497
Nº de valores únicos de perfil_id: 111
Nº de valores únicos de company: 2
Nº de valores únicos de location: 71
Nº de valores únicos de position: 386
Nº de valores vazios de company: 25
Nº de valores vazios de date: 26
Nº de valores vazios de location: 160
Nº de valores vazios de position: 25


In [3]:
experiencia = experiencia[(experiencia.company.notnull()) & (experiencia.position.notnull()) & (experiencia.date.notnull())]
print("Nº de valores únicos de perfil_id:", experiencia.perfil_id.nunique())
print("Nº de registros restantes após remoção dos vazios:", len(experiencia))
print("Nº de valores vazios de company:", experiencia.company.isnull().sum())
print("Nº de valores vazios de date:", experiencia.date.isnull().sum())
print("Nº de valores vazios de location:", experiencia.location.isnull().sum())
print("Nº de valores vazios de position:", experiencia.position.isnull().sum())

print("\nNº de perfil_id excluídos por não possuírem valor de company ou position ou date:")
idx_perfis[~idx_perfis.isin(experiencia.perfil_id.unique())].values


Nº de valores únicos de perfil_id: 110
Nº de registros restantes após remoção dos vazios: 471
Nº de valores vazios de company: 0
Nº de valores vazios de date: 0
Nº de valores vazios de location: 134
Nº de valores vazios de position: 0

Nº de perfil_id excluídos por não possuírem valor de company ou position ou date:


array([108], dtype=int64)

## 05.1 - Tratamento das data

A coluna **date** pode ser quebrada no ano de início (**inicio**) e no ano de fim (**fim**) da experiência. As experiências em aberto, possuem o texto "o momento" que será interpretado como o mês/ano da extração dos dados (09/2017). As experiências em aberto serão marcadas na coluna booleana **atual**, que indicando que o funcionário ainda trabalha na trojan.

In [4]:
experiencia.date = experiencia.date.str.replace('o momento','set de 2017')

ini_fim = experiencia.date.str.split('–')

experiencia['mesano_ini'] = ini_fim.str.get(0).str.strip()
experiencia['mesano_fim'] = ini_fim.str.get(1).str.strip()

#Cria a coluna atual para marcar apenas o emprego em aberto na empresa Trojan
experiencia['atual'] = (experiencia.mesano_fim.str.contains('set de 2017')) & (experiencia.company == 'Trojan Technologies')

#cria coluna para registrar o ano de inicio e fim do trabalho
experiencia['ano_ini'] = experiencia.mesano_ini.str.extract('(\d+)')
experiencia['ano_fim'] = experiencia.mesano_fim.str.extract('(\d+)')

A informação de duração da experiência (**duracao**) é derivada calculando a diferença do fim ao início em meses.

In [5]:
# calcula a duração estimada de cada experiência em meses
def format_mes(mes):
    mes = mes.str.replace('jan','01') \
                .str.replace('fev','02') \
                .str.replace('mar','03') \
                .str.replace('abr','04') \
                .str.replace('mai','05') \
                .str.replace('jun','06') \
                .str.replace('jul','07') \
                .str.replace('ago','08') \
                .str.replace('set','09') \
                .str.replace('out','10') \
                .str.replace('nov','11') \
                .str.replace('dez','12')
    return mes

experiencia.mesano_ini = format_mes(experiencia.mesano_ini)
experiencia.mesano_fim = format_mes(experiencia.mesano_fim)

experiencia.loc[(experiencia.mesano_ini.str.len() == 4), 'mesano_ini'] = '01 de '+experiencia.mesano_ini
experiencia.loc[(experiencia.mesano_fim.str.len() == 4), 'mesano_fim'] = '01 de '+experiencia.mesano_fim 

experiencia['d_ini'] = pd.to_datetime(experiencia.mesano_ini, format='%m de %Y')
experiencia['d_fim'] = pd.to_datetime(experiencia.mesano_fim, format='%m de %Y')

experiencia['duracao_meses'] = experiencia.d_fim.dt.to_period('M') - experiencia.d_ini.dt.to_period('M')+1

experiencia

,company,date,location,position,perfil_id,mesano_ini,mesano_fim,atual,ano_ini,ano_fim,d_ini,d_fim,duracao_meses
0,Trojan Technologies,set de 2011 – set de 2017,"Belo Horizonte e Região, Brasil",CEO and Founder,0,09 de 2011,09 de 2017,True,2011,2017,2011-09-01,2017-09-01,73
6,Any Company,mar de 2016 – set de 2017,NaN,Endeavor Entrepreneur,0,03 de 2016,09 de 2017,False,2016,2017,2016-03-01,2017-09-01,19
7,Any Company,out de 2015 – set de 2017,"Belo Horizonte e Região, Brasil",Investor and Board Member,0,10 de 2015,09 de 2017,False,2015,2017,2015-10-01,2017-09-01,24
8,Any Company,out de 2012 – mai de 2013,"Santiago, Chile",Start-Up Chile Entrepreneur,0,10 de 2012,05 de 2013,False,2012,2013,2012-10-01,2013-05-01,8
9,Any Company,set de 2009 – jan de 2011,"São Paulo e Região, Brasil",Founder and Sales Director,0,09 de 2009,01 de 2011,False,2009,2011,2009-09-01,2011-01-01,17
10,Trojan Technologies,jul de 2017 – set de 2017,"Belo Horizonte e Região, Brasil",Product Manager,1,07 de 2017,09 de 2017,True,2017,2017,2017-07-01,2017-09-01,3
11,Any Company,jan de 2017 – set de 2017,"Belo Horizonte, Minas Gerais",Diretora de Whatever,1,01 de 2017,09 de 2017,False,2017,2017,2017-01-01,2017-09-01,9
12,Trojan Technologies,mar de 2017 – jul de 2017,"Belo Horizonte, Minas Gerais",Coordenadora de Onboarding,1,03 de 2017,07 de 2017,False,2017,2017,2017-03-01,2017-07-01,5
13,Any Company,dez de 2016 – mar de 2017,"Belo Horizonte, Minas Gerais",Coordenadora de aceleração,1,12 de 2016,03 de 2017,False,2016,2017,2016-12-01,2017-03-01,4
14,Any Company,jun de 2016 – jan de 2017,"Belo Horizonte e Região, Brasil",Subcoordenadora de Desenvolvimento de Negócios...,1,06 de 2016,01 de 2017,False,2016,2017,2016-06-01,2017-01-01,8


Algumas informações consolidadas por funcionários podem ser extraídas dessa base. É possível extrair o tempo de carreira na empresa em meses (**tempo_empresa_meses**), o tempo de carreira no mercado em geral em meses (**tempo_mercado_meses**), e também a quantidade de empregos que o funcionário já teve (**qtd_empregos**), o que pode auxiliar em algo na análise. Portanto, obtenha essas informações.

In [6]:
# ----------- Cálculo do tempo_de_mercado_meses ------------ 
#     Os períodos de trabalho para um perfil não são subsequentes. Muitos períodos ocorrem simultaneamente.
#     Para cálculo do tempo de mercado foi utilizada uma estratégia de agrupar períodos sobrepostos em um mesmo período
#     contíno. Após esse agrupamento é realizado o cálculo de tempo de mercado para cada perfil

# Função utilizada para agrupar períodos contínuos e calcular o tempo de mercado em cada período
def calc_t_mercado_periodo(x):
     return pd.Series(dict(
                             perfil_id = x['perfil_id'].min(),
                             d_ini = x['d_ini'].min(), 
                             d_fim = x['d_fim'].max(),
                             tempo_meses = x['d_fim'].max().to_period('M') - x['d_ini'].min().to_period('M') +1
                          )
                     )
    
def calculo_meses(df):
#instancia dataframe que receberá os períodos de trabalho agrupados 
    g_periodos = pd.DataFrame()

#itera sobre cada perfil
    for key, p in df.groupby('perfil_id'):
        p = p.reset_index()

#itera sobre os periodos e verifica se o proximo periodo é sobreposto ao grupo de periodo anterior
#se não for registra, a data que sera usada para agrupar os periodos anteriores que sao sobrepostos 
        fim_grupo = p.loc[0, 'd_fim']
        for l in range(len(p)-1):
            if fim_grupo < p.loc[l+1, 'd_ini']:
                p.loc[l, 'grupo'] = p.loc[l, 'd_fim']

#se a data fim do periodo é maior que a data fim do proximo, mantem a data para comparacao       
            if fim_grupo < p.loc[l+1, 'd_fim']:
                fim_grupo = p.loc[l+1, 'd_fim']

#replica a mesma data final para todos os periodos que são sobrepostos
        l = p.tail(1).index
        p.loc[l,'grupo'] =  p.loc[l,'d_fim']
        p['grupo'] = p.grupo.fillna(method='backfill')

#agrupa os periodos e calcula o tempo de mercado para cada grupo de periodo
        p = p.groupby(['perfil_id','grupo'], as_index=False).apply(calc_t_mercado_periodo)
#adiciona periodos agrupados no dataframe
        g_periodos = pd.concat([g_periodos, p], ignore_index=True)
        
    return g_periodos

In [7]:
#calcula tempo total de mercado por perfil
tempo_mercado = experiencia[['perfil_id','d_ini','d_fim']].sort_values(by=['perfil_id','d_ini'])
tempo_mercado = calculo_meses(tempo_mercado)
tempo_mercado = tempo_mercado.groupby('perfil_id', as_index=False).agg({'tempo_meses':'sum'})
tempo_mercado = tempo_mercado.rename(index=int, columns={'tempo_meses':'tempo_mercado_meses'})

tempo_mercado = tempo_mercado.set_index('perfil_id')

#Confirma valores únicos de perfil_id para tempo de mercado
print("Nº de valores únicos de perfil_id:", tempo_mercado.index.nunique())
tempo_mercado.head(5)

Nº de valores únicos de perfil_id: 110


,tempo_mercado_meses
perfil_id,
0,90
1,16
2,22
3,14
4,17


In [8]:
#calcula tempo_empresa_meses
tempo_empresa = experiencia.loc[(experiencia.company == 'Trojan Technologies'), ['perfil_id','d_ini','d_fim']].sort_values(by=['perfil_id','d_ini'])
tempo_empresa = calculo_meses(tempo_empresa)
tempo_empresa = tempo_empresa.groupby('perfil_id', as_index=False).agg({'tempo_meses':'sum'})
tempo_empresa = tempo_empresa.rename(index=int, columns={'tempo_meses':'tempo_empresa_meses'})

tempo_empresa = tempo_empresa.set_index('perfil_id')

#Confirma valores únicos de perfil_id para tempo de empresa
print("Nº de valores únicos de perfil_id:", tempo_empresa.index.nunique())
tempo_empresa.head(5)

Nº de valores únicos de perfil_id: 110


,tempo_empresa_meses
perfil_id,
0,73
1,7
2,11
3,8
4,15


In [9]:
#calcula quantidade de empregos
qtd_empregos = experiencia.groupby(['perfil_id']).position.count().to_frame()
qtd_empregos = qtd_empregos.rename(index=int, columns={'position':'qtd_empregos'})

#Confirma valores únicos de perfil_id para quantidade de empregos
print("Nº de valores únicos de perfil_id:", qtd_empregos.index.nunique())
qtd_empregos.head(5)

Nº de valores únicos de perfil_id: 110


,qtd_empregos
perfil_id,
0,5
1,5
2,5
3,3
4,5


## 05.2 - Cargo atual na empresa


In [10]:
#Funcao para 
def consolida_cargo_atual(df):
    return pd.Series(dict(
                        data_admissao = df['d_ini'].min(),
                        posicao_atual = ' | '.join(df['position'])
                    )
    )
    
# Alguns perfis acumulam mais de uma posicao atualmente na empresa
# foi utilizada uma funcao lambda para concatenar as posicoes acumulados em uma mesma linha do perfil
cargo_atual = experiencia.loc[experiencia.atual] \
                        .groupby(['perfil_id']) \
                        .apply(consolida_cargo_atual)
cargo_atual = cargo_atual.reset_index()

#Formata data_admissao para ano/mes
cargo_atual['data_admissao'] = cargo_atual['data_admissao'].dt.strftime("%Y-%m")

#Confirma valores únicos de perfil_id para o cargo atual na empresa
print("Nº de valores únicos de perfil_id:", cargo_atual.index.nunique())
cargo_atual.head(7)

Nº de valores únicos de perfil_id: 110


,perfil_id,data_admissao,posicao_atual
0,0,2011-09,CEO and Founder
1,1,2017-07,Product Manager
2,2,2016-11,Full Stack Developer
3,3,2017-08,CRM Analyst
4,4,2017-03,Sales Operations Coordinator
5,5,2016-07,Key Account Manager
6,6,2016-11,Software Engineer and Data Scientist | Data En...


## 05.3 - Setor do Funcionário na empresa

A partir da posicao dos funcionários, é possível descobrir (ou inferir) o setor que eles trabalham.

In [11]:
#Quantidade de valores únicos de cargos atuais na empresa
print("Nº de valores únicos de cargo_atual:",cargo_atual.posicao_atual.nunique())

Nº de valores únicos de cargo_atual: 96


In [12]:
#Tratando os cargos para mesmo padrão: sem acentos, sem espaços, caixa alta
def remove_acentos(text):
    return normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')

cargo_atual['posicao_atual'] = cargo_atual.posicao_atual.apply(remove_acentos)
cargo_atual['posicao_atual'] = cargo_atual.posicao_atual.str.strip()
cargo_atual['posicao_atual'] = cargo_atual.posicao_atual.str.upper()


#Quantidade de valores únicos de cargos após tratamento
print("Nº de valores únicos de cargo_atual:",cargo_atual.posicao_atual.nunique())


Nº de valores únicos de cargo_atual: 94


In [13]:
#exportando dados de cargo atual para inserir as informações de setor
#a análise dos cargos, definição e inserção do departamento será feita em planilha eletrônica
cargos = cargo_atual.groupby('posicao_atual').posicao_atual.count().to_frame()
cargos.to_csv('bases/cargos.csv')

In [18]:
#Ao analisar as posições na empresa, encontramos posições de HEAD
#Dessa forma inferimos os 9 departamentos da empresa, utilizados para classificar as posições 
cargos[cargos.posicao_atual.index.str.contains('HEAD')]

,posicao_atual
posicao_atual,
HEAD OF COMMERCIAL,1
HEAD OF CUSTOMER SUCCESS,1
HEAD OF DATA ANALYTICS,1
HEAD OF IN-STORE OPERATION,1
HEAD OF PEOPLE,1
HEAD OF PRODUCT DESIGN - UXP,1
HEAD OF SALES,1
HEAD OF STRATEGY,1
HEAD OF TROJAN TECHNOLOGIES IN-STORE,1


In [117]:
#após edição dos dados em planilha eletrônica, esses são os departamentos atribuídos à cada posição
cargos_departamentos = pd.read_csv("bases/cargos-departamentos.csv")
cargos_departamentos.head(5)

,posicao_atual,posicao_atual.1,departamento
0,ANALISTA DE ATENDIMENTO,1,CUSTOMER SUCCESS
1,ANALISTA DE CONTROLADORIA,1,STRATEGY
2,ANALISTA DE DP,2,PEOPLE
3,ANALISTA DE MARKETING,1,SALES
4,ANALISTA DE TECNOLOGIA DA INFORMACAO,1,TROJAN TECHNOLOGIES IN-STORE


In [120]:
cargo_atual_departamento = pd.merge(cargo_atual, cargos_departamentos[['posicao_atual','departamento']], how='left', on=['posicao_atual'])
cargo_atual_departamento = cargo_atual_departamento.set_index('perfil_id')
cargo_atual_departamento

,data_admissao,posicao_atual,departamento
perfil_id,,,
0,2011-09,CEO AND FOUNDER,STRATEGY
1,2017-07,PRODUCT MANAGER,TROJAN TECHNOLOGIES IN-STORE
2,2016-11,FULL STACK DEVELOPER,TROJAN TECHNOLOGIES IN-STORE
3,2017-08,CRM ANALYST,CUSTOMER SUCCESS
4,2017-03,SALES OPERATIONS COORDINATOR,SALES
5,2016-07,KEY ACCOUNT MANAGER,CUSTOMER SUCCESS
6,2016-11,SOFTWARE ENGINEER AND DATA SCIENTIST | DATA EN...,DATA ANALYTICS
7,2017-04,APREDIZ ADMINISTRATIVO,STRATEGY
8,2011-01,FOUNDER AND CFO,STRATEGY


In [121]:
#Confirma valores únicos de perfil_id para o cargo atual na empresa
print("Nº de valores únicos de perfil_id:", cargo_atual_departamento.index.nunique())
cargo_atual_departamento.head(5)

Nº de valores únicos de perfil_id: 110


,data_admissao,posicao_atual,departamento
perfil_id,,,
0,2011-09,CEO AND FOUNDER,STRATEGY
1,2017-07,PRODUCT MANAGER,TROJAN TECHNOLOGIES IN-STORE
2,2016-11,FULL STACK DEVELOPER,TROJAN TECHNOLOGIES IN-STORE
3,2017-08,CRM ANALYST,CUSTOMER SUCCESS
4,2017-03,SALES OPERATIONS COORDINATOR,SALES


## 05.4 Salvando Base Tratada

A base tratada será gravada no arquivo **experiencia_trabalho-tratado.csv**.

As informações consolidadas por perfil serão gravadas na base **perfil-experiencia,csv** contendo as colunas: **perfil_id**, **posicao_atual**, **data_admissao**, **departamento**, **tempo_empresa_meses**, **tempo_mercado_meses**, **qtd_empregos**.

In [122]:
# Grava experiencia_trabalho-tratado.csv
experiencia.to_csv('bases/experiencia_trabalho-tratado.csv')

In [123]:
# Grava  perfil-experiencia.csv
perfil_experiencia = pd.concat([cargo_atual_departamento, tempo_empresa, tempo_mercado, qtd_empregos], axis=1, sort=False)
perfil_experiencia.to_csv('bases/perfil-experiencia.csv')

### Navegação:
* [Voltar para a análise](00-analise-dos-dados.ipynb)